In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
((sqrt(2)-I)^3+(sqrt(2)-I)).expand()

-6*I

In [3]:
(sqrt(2)-I).minpoly('xx')

xx^4 - 2*xx^2 + 9

In [4]:
var('xx')
Q.<rad2_minus_i> = NumberField(xx^4 - 2*xx^2 + 9)

In [5]:
ii = ((rad2_minus_i)^3+rad2_minus_i)/(-6)

In [6]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [7]:
e_sost = {e: (-4*ii)*c + d + (-2*ii + 1)*f}
d_sost = {d: ((-64*ii - 48)*c^2 + (64*ii + 128)*c*f + (64*ii + 48)*f^2)/((32*ii - 16)*c + (-32*ii + 16)*f)}

In [8]:
SE1_1 = SE1.subs(e_sost).subs(d_sost)

In [9]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 6.76 s, sys: 198 ms, total: 6.96 s
Wall time: 38 s


576

In [10]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 7.56 s, sys: 197 ms, total: 7.76 s
Wall time: 19.1 s


4

In [11]:
%time conds = SE1_1.find_conditions_for_subfamilies(adm_SE1_1, simm_SE1_1)

CPU times: user 7.4 s, sys: 288 ms, total: 7.69 s
Wall time: 3min 3s


In [13]:
sosts = [solve_linear_system(cond.gens(), [c], [f]) for cond in conds]
cubics = [SE1_1.subs({c:sost[0], f:sost[1]}) for sost in sosts]

In [29]:
%%time
L_sets = list(set([get_permuted_L_set(perm) for perm in SE1_1.find_admissible_permutations()]))
vrs = P.gens()[0:4]
mon = (sum(vrs) ^ 3).monomials()
main_cubic = cubics[0]
equi = [main_cubic.L_set_base] + [None for i in range(len(cubics) - 1)]
for cubic in cubics[1:]:
    for L_set in L_sets:
        proj = cubic.find_projectivity(cubic.L_set_base, L_set)
        eqn = cubic.eqn.subs(change_coordinates(proj))
        if main_cubic.are_cubics_same(eqn):
            equi[cubics.index(cubic)] = L_set
            break

CPU times: user 1min 48s, sys: 320 ms, total: 1min 48s
Wall time: 1min 48s


In [30]:
equi

[('E1', 'G4', 'E2', 'G3', 'E3'),
 ('E1', 'G4', 'E6', 'F16', 'E5'),
 ('F14', 'G4', 'F34', 'F56', 'E2'),
 ('F14', 'G4', 'E5', 'G1', 'F46'),
 ('F14', 'G4', 'E2', 'F26', 'E3'),
 ('F14', 'E1', 'G5', 'F35', 'G2'),
 ('G4', 'F14', 'F35', 'E5', 'E4'),
 ('G4', 'F14', 'F56', 'E6', 'F25'),
 ('E1', 'F14', 'F26', 'G6', 'G1'),
 ('E1', 'F14', 'F56', 'G6', 'F25'),
 ('G4', 'F14', 'G1', 'E2', 'F35'),
 ('G4', 'F14', 'F23', 'E2', 'F36')]

In [31]:
SE1_2 = main_cubic

In [32]:
%time adm_SE1_2 = SE1_2.find_admissible_projectivities()
len(adm_SE1_2)

CPU times: user 3.55 s, sys: 285 ms, total: 3.83 s
Wall time: 21.9 s


576

In [33]:
%time simm_SE1_2 = SE1_2.find_simmetries(adm_SE1_2)
len(simm_SE1_2)

CPU times: user 3.22 s, sys: 164 ms, total: 3.39 s
Wall time: 3.89 s


8

In [34]:
lines_perms = [Permutation(SE1_2.apply_proj_to_lines(simm)).to_permutation_group_element() for simm in simm_SE1_2]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(8, 'C8')

In [36]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()]

[('E1', 'F14', 'E4', 'G5', 'F26', 'F25')]